<a href="https://colab.research.google.com/github/yeswhos/MSc-Project/blob/master/Control%20System/Decision%20Tree/Decision_Tree_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
#filepath = "drive/My Drive/lenses.txt"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# def createDataSet():
#     dataSet = [[1, 1, 'YES'],
#           [1, 1, 'YES'],
#           [1, 0, 'NO'],
#           [0, 1, 'NO'],
#           [0, 1, 'NO']]
#     featNames = ['no surfacing','flippers']
#     return dataSet, featNames


In [ ]:
# def createDataSet():
#     dataSet = [[0, 0, 'Hit it'],
#           [0, 1, 'Hit it'],
#           [0, 2, 'Hit it'],
#           [0, 3, 'Hit it'],
#           [1, 0, 'run away'],
#           [1, 1, 'hit it'],
#           [1, 2, 'hit it'],
#           [1, 3, 'Hit it'],
#           [2, 0, 'run away'],
#           [2, 1, 'run away'],
#           [2, 2, 'Hit it'],
#           [2, 3, 'Hit it'],
#           [3, 0, 'run away'],
#           [3, 1, 'run away'],
#           [3, 2, 'run away'],
#           [3, 3, 'hit it']]
#     featNames = ['Front left Sensor Left','Front Sensor Right', 'line sensor 1', 'line sensor 2', 'line sensor3']
#     return dataSet, featNames

In [4]:
def createDataSet():
    dataSet = [[0, 0, 1, 'Charge'],
          [0, 1, 1, 'Charge'],
          [0, 2, 1, 'Charge'],
          [0, 3, 1, 'Charge'],
          [0, 0, 0, 'No object seeing'],
          [0, 1, 0, 'No object seeing'],
          [0, 2, 0, 'Object seeing'],
          [0, 3, 0, 'Object seeing'],

          [1, 0, 1, 'Charge'],
          [1, 1, 1, 'Charge'],
          [1, 2, 1, 'Charge'],
          [1, 3, 1, 'Charge'],
          [1, 0, 0, 'No object seeing'],
          [1, 1, 0, 'No object seeing'],
          [1, 2, 0, 'Object seeing'],
          [1, 3, 0, 'Object seeing'],

          [2, 0, 1, 'Charge'],
          [2, 1, 1, 'Charge'],
          [2, 2, 1, 'Charge'],
          [2, 3, 1, 'Charge'],
          [2, 0, 0, 'Object seeing'],
          [2, 1, 0, 'Object seeing'],
          [2, 2, 0, 'Object seeing'],
          [2, 3, 0, 'Object seeing'],


          [3, 0, 1, 'Charge'],
          [3, 1, 1, 'Charge'],
          [3, 2, 1, 'Charge'],
          [3, 3, 1, 'Charge'],
          [3, 0, 0, 'Object seeing'],
          [3, 1, 0, 'Object seeing'],
          [3, 2, 0, 'Object seeing'],
          [3, 3, 0, 'Object seeing']]
    featNames = ['Front left Sensor Left','Front Sensor Right', 'Contact']
    return dataSet, featNames

In [5]:
from math import log
 
def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet:
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys(): labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        shannonEnt -= prob * log(prob, 2)
    return shannonEnt


In [6]:
def splitDataSet(dataSet, axis, value):
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]  # chop out axis used for splitting
            reducedFeatVec.extend(featVec[axis + 1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet


In [7]:

dataSet, feats = createDataSet()
print(splitDataSet(dataSet, 0, 1)) #按特征0划分，特征值为1
print(splitDataSet(dataSet, 0, 0))

[[0, 1, 'Charge'], [1, 1, 'Charge'], [2, 1, 'Charge'], [3, 1, 'Charge'], [0, 0, 'No object seeing'], [1, 0, 'No object seeing'], [2, 0, 'Object seeing'], [3, 0, 'Object seeing']]
[[0, 1, 'Charge'], [1, 1, 'Charge'], [2, 1, 'Charge'], [3, 1, 'Charge'], [0, 0, 'No object seeing'], [1, 0, 'No object seeing'], [2, 0, 'Object seeing'], [3, 0, 'Object seeing']]


In [8]:
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1  #每条数据的特征数量
    baseEntropy = calcShannonEnt(dataSet)  #划分前的熵
    bestInfoGain = 0.0;  #记录最高信息增益
    bestFeature = -1  #记录最佳特征
    for i in range(numFeatures):  #遍历每个特征
        featList = [data[i] for data in dataSet]  #把所有数据的该特征值抽出来放到一个list里面
        uniqueVals = set(featList)  #利用set找出该特征所有不同的值
        newEntropy = 0.0
        for value in uniqueVals:  #按这些不同的特征值，分别划分成子数据集
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet) / float(len(dataSet))  #子数据集的权重
            newEntropy += prob * calcShannonEnt(subDataSet)  #子数据集的权重*熵
        infoGain = baseEntropy - newEntropy  #计算信息增益
        if infoGain > bestInfoGain:  #更新最高信息增益和最佳特征
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature  #返回最佳特征的下标


In [9]:
dataSet, feats = createDataSet()
print(chooseBestFeatureToSplit(dataSet))

2


In [10]:
import operator
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys(): classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.iteritems(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]


In [11]:

def createTree(dataSet, featNames):
    classList = [data[-1] for data in dataSet]  #当前数据集的所有标签
    if classList.count(classList[0]) == len(classList):  #如果这堆标签全都一样的话，返回这个标签。
        return classList[0]
    if len(dataSet[0]) == 1:  #如果当前数据集一个特征都不剩了，那就不用再划分下去了
        return majorityCnt(classList)  #直接投票，返回出现次数最多的标签
    bestFeat = chooseBestFeatureToSplit(dataSet)  #选出用于划分的最佳属性
    bestFeatName = featNames[bestFeat]  #最佳属性的属性名称
    myTree = {bestFeatName:{}}  #字典：记录最佳属性对应的标签种类、出现次数情况
    del(featNames[bestFeat])  #在属性名称列表中剔除最佳属性
    featValues = [data[bestFeat] for data in dataSet]  #当前数据集中最佳属性的所有属性值
    uniqueVals = set(featValues) #最佳属性的不同属性值
    for value in uniqueVals:
        subfeatNames = featNames[:]  #去除最佳属性后的属性名称列表
        # 构建最佳属性的值为value的子树
        myTree[bestFeatName][value] = createTree(splitDataSet(dataSet, bestFeat, value),subfeatNames)
    return myTree


In [12]:
dataSet, feats = createDataSet()
theTree = createTree(dataSet, feats)
print(theTree)

{'Contact': {0: {'Front left Sensor Left': {0: {'Front Sensor Right': {0: 'No object seeing', 1: 'No object seeing', 2: 'Object seeing', 3: 'Object seeing'}}, 1: {'Front Sensor Right': {0: 'No object seeing', 1: 'No object seeing', 2: 'Object seeing', 3: 'Object seeing'}}, 2: 'Object seeing', 3: 'Object seeing'}}, 1: 'Charge'}}


In [13]:

def classify(inputTree,featNames,testVec):
    #得换成list
    firstStr = list(inputTree.keys())[0]  #当前树的根节点的特征名称
    secondDict = inputTree[firstStr]  #根节点的所有子节点
    featIndex = featNames.index(firstStr)  #找到根节点特征对应的下标
    key = testVec[featIndex]  #找出待测数据的特征值
    valueOfFeat = secondDict[key]  #拿这个特征值在根节点的子节点中查找，看它是不是叶节点
    if isinstance(valueOfFeat, dict):  #如果不是叶节点
        classLabel = classify(valueOfFeat, featNames, testVec)  #递归地进入下一层节点
    else: classLabel = valueOfFeat  #如果是叶节点：确定待测数据的分类
    return classLabel


In [15]:
dataSet, feats = createDataSet()
feats_copy = feats[:]  #由于createTree函数会改变feats，所以先深复制一份feats
theTree = createTree(dataSet, feats)
# print feats_copy, '\n', theTree
# print classify(theTree, feats_copy, [1,0])
# print classify(theTree, feats_copy, [1,1])
print(feats_copy)
print(theTree)
#print(classify(theTree, feats_copy, [1,0]))

['Front left Sensor Left', 'Front Sensor Right', 'Contact']
{'Contact': {0: {'Front left Sensor Left': {0: {'Front Sensor Right': {0: 'No object seeing', 1: 'No object seeing', 2: 'Object seeing', 3: 'Object seeing'}}, 1: {'Front Sensor Right': {0: 'No object seeing', 1: 'No object seeing', 2: 'Object seeing', 3: 'Object seeing'}}, 2: 'Object seeing', 3: 'Object seeing'}}, 1: 'Charge'}}


In [16]:
def storeTree(inputTree, filename):
    import pickle
    fw = open(filename, 'w')
    pickle.dump(inputTree, fw)
    fw.close()
def grabTree(filename):
    import pickle
    fr = open(filename)
    return pickle.load(fr)

In [17]:
# storeTree(theTree, 'storage.txt')
# grabTree('storage.txt')
theTree

{'Contact': {0: {'Front left Sensor Left': {0: {'Front Sensor Right': {0: 'No object seeing',
      1: 'No object seeing',
      2: 'Object seeing',
      3: 'Object seeing'}},
    1: {'Front Sensor Right': {0: 'No object seeing',
      1: 'No object seeing',
      2: 'Object seeing',
      3: 'Object seeing'}},
    2: 'Object seeing',
    3: 'Object seeing'}},
  1: 'Charge'}}

In [18]:
import json

b = json.dumps(theTree)
print(b)
print(type(b))
# f2 = open('drive/My Drive/tree_json.json', 'w')
# f2.write(b)
# f2.close()

{"Contact": {"0": {"Front left Sensor Left": {"0": {"Front Sensor Right": {"0": "No object seeing", "1": "No object seeing", "2": "Object seeing", "3": "Object seeing"}}, "1": {"Front Sensor Right": {"0": "No object seeing", "1": "No object seeing", "2": "Object seeing", "3": "Object seeing"}}, "2": "Object seeing", "3": "Object seeing"}}, "1": "Charge"}}
<class 'str'>


In [24]:
# -*- coding: UTF-8 -*-
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.externals.six import StringIO
from sklearn import tree
import pandas as pd
import numpy as np
import pydotplus

lenses_pd = pd.DataFrame(theTree)	

dot_data = StringIO()
tree.export_graphviz(theTree, out_file = dot_data,							#绘制决策树
					feature_names = lenses_pd.keys(),
					#class_names = theTree.classes_,
					filled=True, rounded=True,
					special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_pdf("drive/My Drive/tree.pdf")	

TypeError: ignored